In [ ]:
import sys
import copy
import math
import pickle

sys.path.append("../../")

import numpy as np
import torch
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

In [ ]:
# First load the deformed peanuts

peanuts = pickle.load(open("../../data/deformed_peanuts.pkl", 'rb'))

peanuts = [torch.tensor(p[:-1]) for p in peanuts]

template = peanuts[0]
peanuts = peanuts[1:10]

In [ ]:
for p in peanuts:
    plt.plot(p[:, 0].numpy(), p[:, 1].numpy())
plt.show()

In [ ]:
aabb_peanut = dm.Utilities.AABB.build_from_points(template)

density = 30
sigma = 0.4

pts_implicit1 = dm.Utilities.fill_aabb(aabb_peanut, density)
angles = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(a) for a in angles])

In [ ]:
C = torch.ones(pts_implicit1.shape[0], 2, 1, requires_grad=True)
implicit1 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C, sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))

In [ ]:
my_atlas = dm.Models.Atlas(template, [implicit1], [dm.Attachment.VarifoldAttachement([0.4])], len(peanuts), 0.1, other_parameters=[implicit1.C])

In [ ]:
fitter = dm.Models.ModelFittingScipy(my_atlas, 1., 1.)

costs = fitter.fit(peanuts, 50, options={}, log_interval=1)

In [ ]:
template = my_atlas.compute_template()

%matplotlib qt5
ax = plt.subplot()
plt.plot(template[:, 0].numpy(), template[:, 1].numpy())
dm.Utilities.plot_C_ellipse(ax, pts_implicit1, implicit1.C.detach(), scale=0.2)
plt.show()